In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_bayesimp as file

In [2]:
# Args setup
ntrial = 50
n = 100
ntest = 100
d = 5
noise = 0.5
niter = 1000
optimise_mu = [False, False, True,True]
exact = [False, True, False, True]
mc_samples = n
kernel = "gaussian"

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for i in range(len(optimise_mu)):           
    for seed in range(ntrial):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                             optimise_mu = optimise_mu[i], exact = exact[i],
                             mc_samples = mc_samples, kernel = kernel,
                             niter = niter)
            futures += [f]

In [5]:
# Check on futures
futures

[<Future: pending, key: main-8a3de74f098b1da89c6f1f539c7ed4ba>,
 <Future: pending, key: main-e0b2d1379cd4497e909994cc8b742d41>,
 <Future: pending, key: main-26712711d53637f72c5741a1ca9ede8d>,
 <Future: pending, key: main-56f693195cfeb2303a025d46e37eea5e>,
 <Future: pending, key: main-80b6b120f2d7b2f70f73dbbab9305e91>,
 <Future: pending, key: main-957dd274e3a37713afa4e0168b08ed87>,
 <Future: pending, key: main-268363195d0c03981efab3c70e684f2a>,
 <Future: pending, key: main-e478853b96f95df7d4e9788a88813106>,
 <Future: pending, key: main-0239efcf8e792a1c2c94dbd5fb983b96>,
 <Future: pending, key: main-38677a88d7bae5f4b62cb7eb9f30c166>,
 <Future: pending, key: main-c8f1cb54d1f7e69a293ccc457f77dada>,
 <Future: pending, key: main-97e9bcdaa7111e0b7c9a3596a2015309>,
 <Future: pending, key: main-d016d09b6a5724c4fa306a6539bca2cf>,
 <Future: pending, key: main-c5e1bbc25b75171e3850b26a37ebc5fe>,
 <Future: pending, key: main-3e8c0b44feacf6c7a10173c12d8c6e3b>,
 <Future: pending, key: main-1266df96c96

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "ablation_bayesimp_ntrial={0}_n={1}_d={2}_noise={3}_kernel={4}_new.pt".format(ntrial,n,d,noise,kernel))